In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:

!pip install tokenizer transformers scikit-learn pypdf==3.16.0 nltk pandarallel pandas==2.1.0 datasets


In [ ]:
#all imports
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import datasets
import transformers

In [ ]:
resumedf=pd.read_csv("/kaggle/input/resume-dataset/Resume/Resume.csv")
resumedf=resume_data.drop(["Resume_html"],axis=1)

In [ ]:
#getting all text from a pdf
from pypdf import PdfReader

def pdf_text(filePath:str)->str:
    reader = PdfReader(filePath)
    text=""
    for page in reader.pages:
        text+=page.extract_text()
    return text

In [ ]:
#doing text cleaninig and tokenization
from nltk import pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string
import re

# Define English stopwords and punctuation
stop_words_english = set(stopwords.words("english"))
punctuation = set(string.punctuation)

def preprocess_text(text):
    # Convert text to lowercase and remove non-alphabetical characters
    text = re.sub('[^a-zA-Z]', ' ', text.lower())

    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Initialize a list to store extracted features
    extracted_features = []

    # Define POS tags to filter out
    pos_tags_to_filter = ['DT', 'IN', 'TO', 'PRP', 'WP']

    for sent in sentences:
        if any(criteria in sent for criteria in ['skills', 'education']):
            # Tokenize the sentence into words
            words = word_tokenize(sent)

            # Remove stopwords and words with specific POS tags
            filtered_words = [word for word, tag in pos_tag(words) if tag not in pos_tags_to_filter and word not in stop_words_english]

            # Extend the extracted features list
            extracted_features.extend(filtered_words)

    # Join the extracted features into a single string
    return ' '.join(extracted_features)


In [ ]:
progress_bar=tqdm(range(len(resumedf)))
# extract text from csv and get the text from pdf
def process(df):
    #uncomment when using pdf_text function .. done to save time
    id=df['ID']
    category=df['Category']
    text=pdf_text(f"/kaggle/input/resume-dataset/data/data/{category}/{id}.pdf")
   # text=str(df["Resume_str"])
    features=preprocess_text(text)
    df['Feature']=features['feature']
    progress_bar.update(1)
    return df

#applying processing to resume_data
resumedf=resume_data.apply(process,axis=1)
resumedf=resume_data.drop(columns=['Resume_str'])



In [ ]:
#to avoid computing again and save time
#resume_data.to_csv("/path_to_save_processed_resume_data")
resumedf.to_csv("/kaggle/working/resume_data.csv",index=False)

In [ ]:
#resume_data=pd.read_csv("/path_to_save_processed_resume_data")
resumedf=pd.read_csv("/kaggle/working/resume_data.csv")

In [ ]:
resumedf.head()

In [ ]:
resumedf['Category'].value_counts().sort_index().plot(kind="bar",figsize=(12,6))

In [ ]:

num_desc=15

description=pd.read_csv("/kaggle/input/resume-and-job-description/training_data.csv")

description=description[["description","position_title"]][:num_desc]

In [ ]:

description['Features']=description['job_description'].apply(lambda x : preprocess_text(x)['feature'])

In [ ]:
description_df2=description_df2[["description","position_title"]][:num_desc]
description_df2['Features']=description_df2['description'].apply(lambda x : preprocess_text(x)['feature'])


In [ ]:
description_df2.head()

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

# Determine the device (GPU if available, otherwise CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

def get_embeddings(text):
    # Tokenize and encode the input text and move it to the selected device
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

    # Forward pass through the model and get the embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the mean of the embeddings over the tokens
    embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()

    return embeddings


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

job_desc_embeddings = np.array([get_embeddings(desc) for desc in job_description['Features']])
resume_embeddings = np.array([get_embeddings(text) for text in resume_data['Feature']])



In [ ]:
#sqeezing job embeding
resume_embeddings = resume_embeddings.squeeze()
job_desc_embeddings = job_desc_embeddings.squeeze()

print("Resume Embeddings Shape:", resume_embeddings.shape)
print("Job Description Embeddings Shape:", job_desc_embeddings.shape)


In [ ]:
# Initialize a DataFrame to store the results
result_df = pd.DataFrame(columns=['jobId', 'resumeId', 'similarity', 'domainResume', 'domainDesc'])


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame(columns=['job_desc_id', 'resume_id', 'similarity', 'work_domain', 'job_title'])

# Define the number of top-k similar resumes to retrieve
k = 10

# Iterate over job descriptions
for i, job_desc_emb in enumerate(job_desc_embeddings):
    job_desc_id = i
    job_title = job_description['position_title'].iloc[i]

    # Compute cosine similarities between the current job description and all resumes
    similarities = cosine_similarity([job_desc_emb], resume_embeddings)

    # Get the indices of the top-k most similar resumes
    top_k_indices = np.argsort(similarities[0])[::-1][:k]

    # Extract the relevant information and add it to the result DataFrame
    result_df = pd.concat([
        result_df,
        pd.DataFrame({
            'job_desc_id': [job_desc_id] * k,
            'resume_id': resume_data['ID'].iloc[top_k_indices].tolist(),
            'similarity': similarities[0][top_k_indices],
            'work_domain': resume_data['Category'].iloc[top_k_indices].tolist(),
            'job_title': [job_title] * k
        })
    ])

# Sort the results by similarity score (descending)
result_df = result_df.sort_values(by='similarity', ascending=False)


In [ ]:
result_df.head()

In [ ]:
result_group=result_df.groupby("jobId")
result_group

In [ ]:
#example of group data
result_group.get_group(0)

In [ ]:
for i in range(num_desc):
    print()
    print("jobId---cosineSimilarity---domainResume---domainDesc")
    print(result_group.get_group(i).values[0])
    print()